## 1) Loading Data and Packages + Util

In [2]:
import os
from google.colab import files
import shutil

import math
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing_extensions import final
import seaborn as sns
import random

from collections import OrderedDict

Load data from local folder

In [3]:
os.makedirs('/content/data', exist_ok=True)
destination_folder = '/content/data'
uploaded = files.upload()
for filename in uploaded.keys():
    shutil.move(filename, os.path.join(destination_folder, filename))

Saving full_fb_data.csv to full_fb_data.csv


Setup fitbit_data

In [4]:
full_fb_data = pd.read_csv('/content/data/full_fb_data.csv')

In [5]:
survey_features = ['diet', 'medication', 'symptoms']

In [6]:
full_fb_data['date'] = pd.to_datetime(full_fb_data['date'])
full_fb_data = full_fb_data.sort_values(by=['fitbit_user_id', 'date'])

# Shift survey columns
for col in survey_features:
  full_fb_data[col] = full_fb_data.groupby('fitbit_user_id')[col].shift(-1)

# Set remaining nan visit columns to 0
full_fb_data['visit_day'] = full_fb_data['visit_day'].fillna(0)

In [7]:
full_fb_data['has_visit'] = 0

for user_id, group in full_fb_data.groupby('fitbit_user_id'):
    if group['visit_day'].sum() != 0:
        full_fb_data.loc[group.index, 'has_visit'] = 1

In [8]:
def extract_days_before_visit(df):
    output_rows = []

    for user_id, group in df.groupby('fitbit_user_id'):
        group = group.reset_index(drop=True)
        visit_indices = group.index[group['visit_day'] == 1].tolist()

        if not visit_indices:
            continue  # Skip users with no visits

        first_visit_idx = visit_indices[0]
        visit_day = group.loc[first_visit_idx, 'days']

        # Get all days before the first visit (exclude visit day itself)
        pre_visit_data = group[group['days'] < visit_day].copy()

        # Add relative day to visit
        pre_visit_data['days_to_visit'] = pre_visit_data['days'] - visit_day
        pre_visit_data['fitbit_user_id'] = user_id

        output_rows.append(pre_visit_data)

    result_df = pd.concat(output_rows, axis=0).reset_index(drop=True)

    return result_df

In [9]:
visit_df = extract_days_before_visit(full_fb_data)
no_visit_df = full_fb_data[full_fb_data['has_visit'] == 0]